# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 36 new papers today
          12 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.00679


extracting tarball to tmp_2302.00679... done.
Retrieving document from  https://arxiv.org/e-print/2302.00699


extracting tarball to tmp_2302.00699...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.00718


...

extracting tarball to tmp_2302.01239...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.01245


extracting tarball to tmp_2302.01245...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.01276


/tmp/ipykernel_2070/4030337529.py:34: LatexWarning: 2302.01276 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00679-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00679) | **The Density Parity Model for the Evolution of the Galaxy Inner Spin  Alignments with the Cosmic Web**  |
|| Jun-Sung Moon, <mark>Jounghun Lee</mark> |
|*Appeared on*| *2023-02-03*|
|*Comments*| *submitted for publication in ApJ, 16 figures*|
|**Abstract**| We develop a new model within which the radius-dependent transition of the galaxy inner spins with respect to the cosmic web and the variation of the transition threshold radius ($r_{\rm th}$) with galaxy mass ($M_{\rm vir}$), smoothing scale ($r_{f}$), and redshift ($z$) can be coherently explained. The key tenet of this model is that the competition between the pressure effect of the inner mass and the compression effect of the local tidal field determines which principal direction of the tidal field the inner spins are aligned with. If the former predominates, then only the tidal torques turn on, resulting in the alignments of the inner spins with the intermediate principal axes of the tidal field. Otherwise, the galaxy spins acquire a tendency to be aligned with the shortest axes of the galaxy shapes, which is in the major principal directions of the tidal field. Quantifying the two effects in terms of the mean squared densities, we make a purely analytical prediction for $r_{\rm th}(M_{\rm vir}, z, r_{f})$. Testing this model against the numerical results from a high-resolution simulation in the redshift range of $0\le z\le 3$ on the galactic mass scale of $11.8\le \log M_{\rm vir}/(h^{-1}M_{\odot})\le 12.6$ for two different cases of $r_{f}/(h^{-1}{\rm Mpc})=0.5$ and $1$, we find excellent agreements of the model predictions with the numerical results. It is also shown that this model naturally predicts the alignments between the inner spins of the present galaxies with the principal directions of the high-$z$ tidal field at the progenitors' locations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00699-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00699) | **An Earth-sized Planet around an M5 Dwarf Star at 22 pc**  |
|| Teruyuki Hirano, et al. -- incl., <mark>John H. Livingston</mark>, <mark>Alexis M. S. Smith</mark>, <mark>Akitoshi Ueda</mark> |
|*Appeared on*| *2023-02-03*|
|*Comments*| *20 pages, 12 figures, accepted for publication in AJ*|
|**Abstract**| We report on the discovery of an Earth-sized transiting planet ($R_p=1.015\pm0.051\,R_\oplus$) in a $P=4.02$ day orbit around K2-415 (EPIC 211414619), an M5V star at 22 pc. The planet candidate was first identified by analyzing the light curve data by the K2 mission, and is here shown to exist in the most recent data from TESS. Combining the light curves with the data secured by our follow-up observations including high-resolution imaging and near infrared spectroscopy with IRD, we rule out false positive scenarios, finding a low false positive probability of $2\times 10^{-4}$. Based on IRD's radial velocities of K2-415, which were sparsely taken over three years, we obtain the planet mass of $3.0\pm 2.7\,M_\oplus$ ($M_p<7.5\,M_\oplus$ at $95\,\%$ confidence) for K2-415b. Being one of the lowest mass stars ($\approx 0.16\,M_\odot$) known to host an Earth-sized transiting planet, K2-415 will be an interesting target for further follow-up observations, including additional radial velocity monitoring and transit spectroscopy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.00718-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.00718) | **A novel approach to detect line emission under high background in  high-resolution X-ray spectra**  |
|| <mark>Xiangyu Zhang</mark>, Sara Algeri, Vinay Kashyap, Margarita Karovska |
|*Appeared on*| *2023-02-03*|
|*Comments*| *Accepted by Monthly Notices of the Royal Astronomical Society*|
|**Abstract**| We develop a novel statistical approach to identify emission features or set upper limits in high-resolution spectra in the presence of high background. The method relies on detecting differences from the background using smooth tests and using classical likelihood ratio tests to characterise known shapes like emission lines. We perform signal detection or place upper limits on line fluxes while accounting for the problem of multiple comparisons. We illustrate the method by applying it to a Chandra LETGS+HRC-S observation of symbiotic star RT Cru, successfully detecting previously known features like the Fe line emission in the 6-7 keV range and the Iridium-edge due to the mirror coating on Chandra. We search for thermal emission lines from Ne X, Fe XVII, O VIII, and O VII, but do not detect them, and place upper limits on their intensities consistent with a $\approx$1 keV plasma. We serendipitously detect a line at 16.93 $\unicode{x212B}$ that we attribute to photoionisation or a reflection component. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01232-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01232) | **The Design Strain Sensitivity of the Schenberg Spherical Resonant  Antenna for Gravitational Waves**  |
|| <mark>V. Liccardo</mark>, et al. |
|*Appeared on*| *2023-02-03*|
|*Comments*| **|
|**Abstract**| The main purpose of this study is to review the Schenberg resonant antenna transfer function and to recalculate the antenna design strain sensitivity for gravitational waves. We consider the spherical antenna with six transducers in the semi dodecahedral configuration. When coupled to the antenna, the transducer-sphere system will work as a mass-spring system with three masses. The first one is the antenna effective mass for each quadrupole mode, the second one is the mass of the mechanical structure of the transducer first mechanical mode and the third one is the effective mass of the transducer membrane that makes one of the transducer microwave cavity walls. All the calculations are done for the degenerate (all the sphere quadrupole mode frequencies equal) and non-degenerate sphere cases. We have come to the conclusion that the 'ultimate' sensitivity of an advanced version of Schenberg antenna (aSchenberg) is around the standard quantum limit (although the parametric transducers used could, in principle, surpass this limit). However, this sensitivity, in the frequency range where Schenberg operates, has already been achieved by the two aLIGOs in the O3 run, therefore, the only reasonable justification for remounting the Schenberg antenna and trying to place it in the sensitivity of the standard quantum limit would be to detect gravitational waves with another physical principle, different from the one used by laser interferometers. This other physical principle would be the absorption of the gravitational wave energy by a resonant mass like Schenberg. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01239-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01239) | **The Solar System Notification Alert Processing System (SNAPS): Design,  Architecture, and First Data Release (SNAPShot1)**  |
|| David E. Trilling, et al. -- incl., <mark>Daniel Kramer</mark> |
|*Appeared on*| *2023-02-03*|
|*Comments*| *AJ, in press*|
|**Abstract**| We present here the design, architecture, and first data release for the Solar System Notification Alert Processing System (SNAPS). SNAPS is a Solar System broker that ingests alert data from all-sky surveys. At present, we ingest data from the Zwicky Transient Facility (ZTF) public survey, and we will ingest data from the forthcoming Legacy Survey of Space and Time (LSST) when it comes online. SNAPS is an official LSST downstream broker. In this paper we present the SNAPS design goals and requirements. We describe the details of our automatic pipeline processing in which physical properties of asteroids are derived. We present SNAPShot1, our first data release, which contains 5,458,459 observations of 31,693 asteroids observed by ZTF from July, 2018, through May, 2020. By comparing a number of derived properties for this ensemble to previously published results for overlapping objects we show that our automatic processing is highly reliable. We present a short list of science results, among many that will be enabled by our SNAPS catalog: (1) we demonstrate that there are no known asteroids with very short periods and high amplitudes, which clearly indicates that in general asteroids in the size range 0.3--20 km are strengthless; (2) we find no difference in the period distributions of Jupiter Trojan asteroids, implying that the L4 and L5 cloud have different shape distributions; and (3) we highlight several individual asteroids of interest. Finally, we describe future work for SNAPS and our ability to operate at LSST scale. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01245-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01245) | **Whistler waves generated inside magnetic dips in the young solar wind:  observations of the Search-Coil Magnetometer on board Parker Solar Probe**  |
|| C. Froment, et al. -- incl., <mark>F. S. Mozer</mark> |
|*Appeared on*| *2023-02-03*|
|*Comments*| *15 pages, 14 figures, recommended for publication in A&A*|
|**Abstract**| Context. Whistler waves are electromagnetic waves produced by electron-driven instabilities, that in turn can reshape the electron distributions via wave-particle interactions. In the solar wind, they are one of the main candidates for explaining the scattering of the strahl electron population into the halo at increasing radial distances from the Sun and for subsequently regulating the solar wind heat flux. However, it is unclear what type of instability dominates to drive whistlers in the solar wind. Aims. Our goal is to study whistler wave parameters in the young solar wind sampled by Parker Solar Probe (PSP). The wave normal angle (WNA) in particular is a key parameter to discriminate between the generation mechanisms of these waves. Methods. We analyze the cross-spectral matrices of magnetic fieldfluctuations measured by the Search-Coil Magnetometer (SCM) and processed by the Digital Fields Board (DFB) from the FIELDS suite during PSP's first perihelion. Results. Among the 2701 wave packets detected in the cross spectra, namely individual bins in time and frequency, most were quasi-parallel to the background magnetic field but a significant part (3%) of observed waves had oblique (> 45{\deg}) WNA. The validation analysis conducted with the time-series waveforms reveal that this percentage is a lower limit. Moreover, we find that about 64% of the whistler waves detected in the spectra are associated with at least one magnetic dip. Conclusions. We conclude that magnetic dips provides favorable conditions for the generation of whistler waves. We hypothesize that the whistlers detected in magnetic dips are locally generated by the thermal anisotropy as quasi-parallel and can gain obliqueness during their propagation. We finally discuss the implication of our results for the scattering of the strahl in the solar wind. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01168-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01168) | **OGLE-2016-BLG-1195Lb: A Sub-Neptune Beyond the Snow Line of an M-dwarf  Confirmed by Keck AO**  |
|| Aikaterini Vandorou, et al. -- incl., <mark>Lisa Dang</mark> |
|*Appeared on*| *2023-02-03*|
|*Comments*| *Submitted to AJ*|
|**Abstract**| We present the analysis of high resolution follow-up observations of OGLE-2016-BLG-1195 using Keck, four years after the event's peak. We find the lens system to be at $D_L = 6.87\pm 0.65$ kpc and comprised of a $M_{\rm p} = 9.91\pm 1.61\ M_{\rm Earth}$ planet, orbiting an M-dwarf, $M_{\rm L} = 0.57\pm 0.06\ M_{\odot}$, beyond the snow line, with a projected separation of $r_\perp=2.62\pm 0.28$ AU. Our results are consistent with the discovery paper, which reports values with 1-sigma uncertainties based on a single mass-distance constraint from finite source effects. However, both the discovery paper and our follow-up results disagree with the analysis of a different group that also present the planetary signal detection. The latter utilizes Spitzer photometry to measure a parallax signal. Combined with finite source effects, they claim to measure the mass and distance of the system to much greater accuracy, suggesting that it is composed of an Earth-mass planet orbiting an ultracool dwarf. Their parallax signal though is improbable since it suggests a lens star in the disk moving perpendicular to disk rotation. Moreover, parallaxes are known to be affected by systematic errors in the photometry. Therefore, we reanalyze the Spitzer photometry for this event and conclude that the parallax signal is not significantly greater than the instrumental noise, and is likely affected by systematic errors in the photometric data. The results of this paper act as a cautionary tale that conclusions of analyses that rely heavily on low signal-to-noise Spitzer photometric data, can be misleading. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.01276-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.01276) | **Nova explosions -- The fascinating gamma-ray emitting recurrent nova RS  Ophiuchi**  |
|| Vincent Tatischeff, <mark>Margarita Hernanz</mark> |
|*Appeared on*| *2023-02-03*|
|*Comments*| *8 pages, 3 figures, proceedings of the 7th Heidelberg International Symposium on High-Energy Gamma-Ray Astronomy (Gamma2022) in Barcelona (Spain) on 4-8 July 2022*|
|**Abstract**| Classical and recurrent nova explosions occur on top of white dwarfs accreting H-rich matter from a companion main sequence or red giant star, in a close binary system. In the recent years, since the launch of the Fermi gamma-ray satellite by NASA in 2008, several novae have been detected by Fermi/LAT (LAT: Large Area Telescope) in high-energy (HE) gamma rays, with energies larger than 100 MeV. This emission is known to be related to the acceleration of particles in the internal and/or external shocks occurring early after the thermonuclear nova explosion. However, very-high-energy (VHE) gamma-rays, with energies larger than 100 GeV, produced as a consequence of nova explosions have only been discovered very recently, in the recurrent nova RS Oph, that had an outburst in August 2021. These require the acceleration of protons, and not only of electrons; this was in fact predicted theoretically - based in observations at other wavelengths - in the previous eruption of RS Oph, in 2006, but has not been confirmed observationally until now. We review the origin of the different types of gamma-ray emission in novae and highlight the relevance of the recent VHE gamma-ray emission discoveries for the nova theory, mainly in the field of the mass ejection and the associated particle (electrons and protons) acceleration processes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

188  publications in the last 7 days.
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers